In [5]:
import numpy as np
import tensorflow as tf
import os
import csv

from common import dataset_utils

# Preparación del conjunto de datos
Generación del conjunto de entrenamiento, validación y pruebas 

In [3]:
!mv dataset/train.csv dataset/data.csv

In [7]:
data_file_name = 'dataset/data.csv'
data_file_path = os.path.join(os.getcwd(), data_file_name)

In [8]:
# hay 55423855 registros
no_samples = dataset_utils.get_number_samples(data_file_path)
print(no_samples)

# data_field_names = ['key', 'fare_amount', 'pickup_datetime', 'pickup_longitude', 'pickup_latitude', 
#                    'dropoff_longitude', 'dropoff_latitude, passenger_count']
# with open(data_file_path) as data_file:
#     data_file_reader = csv.DictReader(data_file, fieldnames=data_field_names)
#     next(data_file_reader, None)
#     for entry in data_file_reader:
#         print()

55423855


In [12]:
!head {data_file_path}

In [15]:
LABEL_COLUMN = 'fare_amount'

In [20]:
tf.data.experimental.ma
def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5, 
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  
  return dataset

def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

In [21]:
raw_data = get_dataset(data_file_path)
show_batch(raw_data)



key                 : [b'2013-07-24 18:16:00.00000047' b'2014-06-02 08:31:14.0000001'
 b'2014-05-07 09:28:26.0000005' b'2012-07-24 20:28:14.0000004'
 b'2012-01-26 00:26:00.00000067']
pickup_datetime     : [b'2013-07-24 18:16:00 UTC' b'2014-06-02 08:31:14 UTC'
 b'2014-05-07 09:28:26 UTC' b'2012-07-24 20:28:14 UTC'
 b'2012-01-26 00:26:00 UTC']
pickup_longitude    : [-73.968285 -73.966415 -73.99901  -73.98551  -73.98775 ]
pickup_latitude     : [40.75536  40.770348 40.734505 40.74112  40.72944 ]
dropoff_longitude   : [-74.000626 -73.95553  -73.991234 -74.00323  -73.9939  ]
dropoff_latitude    : [40.730137 40.76583  40.73851  40.726883 40.741264]
passenger_count     : [5 1 1 1 1]
